# Cryptography (CC4017) -- Week 11

## Exercise 1

In the context of PKI

(a) Describe what is the accepted procedure of a client when receives a public-key certificate
from a web server?

>1. **Receive Certificate**: The client receives the public-key certificate from the web server during the SSL/TLS handshake.
>
>2. **Verify Certificate Chain**: The client verifies the certificate chain up to a trusted root certificate authority (CA). This involves checking each certificate in the chain to ensure it is signed by a trusted CA.
>
>3. **Check Validity Period**: The client checks the certificate's validity period to ensure it is currently valid (not expired or not yet valid).
>
>4. **Check Revocation Status**: The client checks if the certificate has been revoked. This can be done using Certificate Revocation Lists (CRLs) or the Online Certificate Status Protocol (OCSP).
>
>5. **Verify Domain Name**: The client verifies that the domain name in the certificate matches the domain name of the web server.
>
>6. **Establish Secure Connection**: If all checks pass, the client establishes a secure connection with the web server using the public key in the certificate
>

(b) Describe the process of certificate revocation and what are the possible reasons to apply
it.

>**Process of Certificate Revocation**:
>
>1. **Revocation Request**: The certificate owner or the CA identifies the need to revoke a certificate.
>
>2. **Update CRL/OCSP**: The CA updates its Certificate Revocation List (CRL) or Online Certificate Status Protocol (OCSP) responder to include the revoked certificate.
>
>3. **Distribution**: The updated CRL is distributed to clients, or the OCSP responder is queried by clients to check the revocation status.
>
>4. **Client Check**: Clients check the CRL or query the OCSP responder during the certificate verification process to ensure the certificate has not been revoked.
>
>**Possible Reasons for Certificate Revocation**:
>
>1. **Compromise of Private Key**: If the private key associated with the certificate is compromised, the certificate must be revoked to prevent misuse.
>
>2. **CA Compromise**: If the CA that issued the certificate is compromised, all certificates issued by that CA may need to be revoked.
>
>3. **Change of Information**: If the information in the certificate (e.g., domain name, organization name) changes, the certificate may be revoked and reissued with the updated information.
>
>4. **Violation of CA Policy**: If the certificate owner violates the CA's policies or terms of service, the CA may revoke the certificate.
>
> 5. **Certificate Expiry**: Although not a revocation reason per se, expired certificates are no longer valid and should not be used.
>

## Exercise 2

In a Pailier’s scheme chanel, with private key n = 620496404349687915307910174617, we intercepted the cyphered message 

c = 358624662650643040547102063483144791182626860568435345308004. 

Can you recover the original plaintext?

In [1]:
from sympy import factorint

# The large number to be factorized
large_number =  620496404349687915307910174617

# Perform prime factorization
factors = factorint(large_number)

# Display the results
print("Prime factors and their multiplicities:")
for prime, multiplicity in factors.items():
    print(f"{prime}^{multiplicity}")

Prime factors and their multiplicities:
802829923639097^1
772886493240161^1


In [3]:
from math import gcd 
from sympy import lcm, mod_inverse
# Let n = 802829923639097 x 772886493240161 = 620496404349687915307910174617

def lagrange_function(x,n):
    return (x - 1) // n

def decrypt_paillier(c,n,p,q):
    # Compute n^2
    n_squared = n*n

    # Compute λ = lcm(p-1, q-1)
    lambda_value = int(lcm(p - 1, q - 1))
    
    # Compute g^λ mod n^2 (assuming g = n + 1 as typical)

    g = n + 1
    g_lambda = pow(g, lambda_value, n_squared)

    # Compute L(g^λ mod n^2)
    l_g_lambda = lagrange_function(g_lambda, n)

    # Compute c^λ mod n^2
    c_lambda = pow(c, lambda_value, n_squared)

    # Compute L(c^λ mod n^2)
    l_c_lambda = lagrange_function(c_lambda, n)

    # Compute plaintext m
    m = (l_c_lambda * mod_inverse(l_g_lambda, n)) % n
    return m

# Given values
n = 620496404349687915307910174617
c = 358624662650643040547102063483144791182626860568435345308004
p = 802829923639097
q = 772886493240161

# Decrypt the ciphertext
m = decrypt_paillier(c,n,p,q)
print(f"The decrypted message is: {m}")


The decrypted message is: 455667


## Exercice 3

Write the python/Sage procedures that behave as follows:

**genPrivate(sz)** that outputs a triple (n, p, q) in the conditions to be used as (n) the public key of a Paillier’s scheme and n = pq, being p, q primes of size sz bits;

**voteYes(fileName,n)** that append to file fileName a vote yes (=1) using Paillier’s scheme and public key n.

**voteNo(fileName,n)** that append to file fileName a vote no (=0) using Paillier’s scheme and public key n.

**getResults(fileName,n,phi)** that prints the result of the polling written in fileName being n the public key used and phi the Euler’s totient value corresponding to the public key.

In [4]:
import random
from Crypto.Util import number

def genPrivate(sz):
    p = number.getPrime(sz)
    q = number.getPrime(sz)
    n = p * q
    return (n, p, q)

def encrypt_vote(vote, n):
    r = random.randint(1, n-1)
    while number.GCD(r, n) != 1:
        r = random.randint(1, n-1)
    return (pow(n+1, vote, n*n) * pow(r, n, n*n)) % (n*n)

def voteYes(fileName, n):
    vote = 1
    encrypted_vote = encrypt_vote(vote, n)
    with open(fileName, 'a') as file:
        file.write(f"{encrypted_vote}\n")

def voteNo(fileName, n):
    vote = 0
    encrypted_vote = encrypt_vote(vote, n)
    with open(fileName, 'a') as file:
        file.write(f"{encrypted_vote}\n")

def decrypt_vote(encrypted_vote, n, phi):
    n_sq = n * n
    x = pow(encrypted_vote, phi, n_sq) - 1
    return (x // n) % n

def getResults(fileName, n, phi):
    with open(fileName, 'r') as file:
        votes = file.readlines()
    
    total_votes = 0
    for vote in votes:
        encrypted_vote = int(vote.strip())
        decrypted_vote = decrypt_vote(encrypted_vote, n, phi)
        total_votes += decrypted_vote
    
    print(f"Total votes: {total_votes}")

# Example usage:
#n, p, q = genPrivate(512)
#phi = (p-1)*(q-1)
#voteYes('votes.txt', n)
#voteNo('votes.txt', n)
#getResults('votes.txt', n, phi)

Total votes: 62700007335836519123453141565862176080221543437523990551738678125884865041437707214596884951517385258745600467806117804291769386307728223443525464086562593578283300671228700664412225021345748029567822811755412588371981268414872965675549675783299490685640213106690085611537706684537098526098855616102614751676
